In [ ]:
import os
import sys

sys.path.append('../')

import h5py
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numba
import numpy as np
import torch
from numpy.fft import fft, fft2, fftshift, ifft, ifft2, ifftshift
from tqdm import tqdm

from kwave.ktransducer import *
from kwave.utils import *
from models.ADMM import ADMM_Batched
from models.APACT import APACT
from models.WienerNet import Wiener_Batched
from utils.data import load_mat
from utils.simulations import PSF, get_water_SoS, wavefront_real
from utils.utils_torch import get_fourier_coord
from utils.visualization import normalize, standardize
from utils.reconstruction import *

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_path = '../data/'

### Load in Phantom Data

In [ ]:
sinogram = load_mat(os.path.join(data_path, 'LEAF_PHANTOM.mat'))\

plt.subplots(figsize=(16,5))
plt.imshow(sinogram, cmap='gray')
plt.show()

### Load and Deconvolve EIR

In [ ]:
EIR = load_mat(os.path.join(data_path, 'EIR_TUNGSTEN.mat'))

time = np.linspace(0, 2000, 2000)
plt.figure(figsize=(16,4))
plt.plot(time, np.squeeze(EIR[:2000]))
plt.xlim(0, 2000)
plt.tight_layout()

In [ ]:
def deconvolve_sinogram(sinogram, EIR, t0):
    """Correct EIR phase of the sinogram.

    Args:
        sinogram (`numpy.array`): _description_
        EIR (`numpy.array`): EIR of the transducer.
        t0 (`int`): Time index of the EIR peak.

    Returns:
        `numpy.array`: The corrected sinogrm.
    """
    delta = np.zeros_like(EIR)
    delta[0, t0] = 1
    delta_ft = fft(delta, axis=1)
    EIR_ft = fft(EIR, axis=1)
    Sinogram_ft = fft(sinogram, axis=1)
    Sinogram_ft *= np.exp(1j * (np.angle(delta_ft) - np.angle(EIR_ft)))
    
    return np.real((ifft(Sinogram_ft, axis=1)))

t0 = np.argmax(EIR)    
sinogram_deconv = deconvolve_sinogram(sinogram, EIR, t0)
plt.subplots(figsize=(16,5))
plt.imshow(sinogram_deconv, cmap='gray')
plt.show()

In [ ]:
Nx, Ny = 800, 800
R_ring = 0.05   # Radius of the ring array [m].
dx, dy = 4e-5, 4e-5
kgrid = kWaveGrid([Nx, Ny], [dx, dy])

In [ ]:
R = 12.5e-3 # Radius to center [m].
l = 3.2e-3 # Patch size [m].
T = 26.0 # Temperature of water [C].
v0 = get_water_SoS(T) # Background SoS [m/s].
v1 = 1519 # Avergae SoS in tissue [m/s].

print(v0, v1)

Load ring error.

In [ ]:
data = h5py.File(os.path.join(data_path, 'RING_ERROR_NEW.mat'))
ring_error, transducer_theta = np.array(data['e'])[:,0], np.array(data['transducer_theta'])[:,0]
print(ring_error.shape, transducer_theta.shape)

idx = np.arange(0, 512, 1)
ring_error = np.interp(idx, np.arange(0, 512, 2), ring_error) # Upsample ring error.
plt.figure(figsize=(16,4))
plt.plot(idx, ring_error)
plt.tight_layout()

In [ ]:
# @numba.jit(nopython=True) 
# def delay_and_sum(R_ring, T_sample, V_sound, sinogram, x_vec, y_vec, d_delay=0, ring_error=0):
#     """Generate a 2D Delay And Sum recontructed PACT image of ring transducer array. This function is accelerated by `numba.jit` on a GPU.

#     Args:
#         R_ring (`float`): The R_ring [m] of the ring transducer array.
#         T_sample (`float`): Sample time interval [s] of the signals.
#         V_sound (`float`): The sound speed [m/s] used for Delay And Sum recontruction.
#         sinogram (`numpy.ndarray`): A 2D array and each column of it is the signal recievde by one transducer. The nummber of transducers should be the number of columns. The transducers should be evenly distributed on a circle in counterclockwise arrangement and the first column correspond to the transducer in the dirrection `2pi/N` in the first quartile. The first sample should be at time 0 when the photoacoustic effect happens.
#         x_vec (`numpy.ndarray`): The vector [m] defining the x coordinates of the grid points on which the recontruction is done. The values in the vector should be unifromly-spaced in ascending order. The origin of the cartesian coordinate system is the center of the ring array.
#         y_vec (`numpy.ndarray`): The vector [m] defining the y coordinates of the grid points on which the recontruction is done. The values in the vector should be unifromly-spaced in ascending order. The origin of the cartesian coordinate system is the center of the ring array.
#         d_delay (`float`): The delay distance [m] of the signals used in DAS. The default value is 0.
#         ring_error (`numpy.ndarray`): The radial displacement error of the transducers. The default value is 0.

#     Returns:
#         `numpy.ndarray`: A 2D array of size `(len(y_vec), len(x_vec))`. `Image[t, s]` is the recontructed photoacoustic amplitude at the grid point `(x_vec[s], y_vec[t])`.
#     """

#     N_transducer = Sinogram.shape[0]
#     Image = np.zeros((len(x_vec), len(y_vec)))
#     delta_angle = 2*np.pi / N_transducer
#     angle_transducer = delta_angle * (np.arange(N_transducer,) + 1)
    
#     R_ring = R_ring #- 40e-6 - 120e-6 * np.cos(2 * angle_transducer) + 5e-5 # Ring Array Correction 

#     x_transducer = R_ring * np.sin(angle_transducer - np.pi)
#     y_transducer = R_ring * np.cos(angle_transducer - np.pi)
    
#     related_data = np.zeros((N_transducer,))
    
#     for s in range(len(x_vec)):
#         for t in range(len(y_vec)):
#             distance_to_transducer = np.sqrt((x_transducer - x_vec[s])**2 + (y_transducer - y_vec[t])**2) - d_delay + ring_error
#             for k in range(N_transducer):
#                 id = floor(distance_to_transducer[k] / (V_sound * T_sample))
#                 if id > sinogram.shape[1] or id < 0:
#                     related_data[k] = 0
#                 else:
#                     related_data[k] = sinogram[k, id]
#             Image[t, s] = related_data.mean()
#     return Image

### Conventional DAS

In [ ]:
baseline = delay_and_sum(R_ring=0.05,
                         T_sample=1/(40e6),
                         V_sound=1505.0,
                         sinogram=sinogram_deconv[:,46:],
                         x_vec=kgrid.x_vec[:],
                         y_vec=kgrid.y_vec[:],
                         d_delay=0)
img_size = 560
x_c, y_c = 425, 360 # x_c, y_c = center(baseline)
baseline = baseline[x_c-img_size//2:+x_c+img_size//2, y_c-img_size//2:y_c+img_size//2]

In [ ]:
fig = plt.figure(figsize=(16,6))
ax1 = plt.subplot(1,2,1)
plt.imshow(baseline, cmap='gray')
cax = fig.add_axes([ax1.get_position().x1+0.008, ax1.get_position().y0, 0.015, ax1.get_position().height])
cb = plt.colorbar(cax=cax)
cb.ax.tick_params(labelsize=10)
plt.title('Image', fontsize=12)

ax2 = plt.subplot(1,2,2)
plt.hist(baseline.ravel(), bins=150)
plt.title('Histogram', fontsize=12)

### APACT

In [ ]:
delays_apact = np.arange(-8e-4, 8e-4, 2e-4)
recons = []
for d_delay in delays_apact:
    recon = delay_and_sum(R_ring=0.05,
                          T_sample=1/(40e6), 
                          V_sound=v0,
                          sinogram=sinogram_deconv[:,46:],
                          x_vec=kgrid.x_vec[:],
                          y_vec=kgrid.y_vec[:],
                          d_delay=d_delay)
    recons.append(recon)
recons = np.array(recons)
recons = recons[:,x_c-img_size//2:x_c+img_size//2, y_c-img_size//2:y_c+img_size//2]

In [ ]:
l = 3.2e-3        
obss = []
recons = np.array(recons)
for i in range(25):
    for j in range(25):
        obs = torch.Tensor(recons[:,20*i:20*i+80, 20*j:20*j+80])
        obss.append(obs)

In [ ]:
def gaussian_kernel(sigma, size):
    function = lambda x,y: np.exp(-((x-(size-1)/2)**2 + (y-(size-1)/2)**2) / (2*(sigma**2)))
    kernel = np.fromfunction(function, (size, size), dtype=float)
    return kernel / np.sum(kernel)

fwhm = 1.5e-3 # [m]
sigma = fwhm / 4e-5 / np.sqrt(2*np.log(2))
gaussian_window = gaussian_kernel(sigma, 80)

In [ ]:
model = APACT(delays=delays_apact, step=4e-5)
# model = DataParallel(model)
model.to(device)
model.eval()

rec_full_apact = np.zeros_like(recons[0])
for i in tqdm(range(25)):
    for j in tqdm(range(25)):
        obs = obss[i*25+j] * gaussian_window
        obs = obs.unsqueeze(0).to(device)
        with torch.no_grad():
            rec, _, _, _ = model(obs)
        rec = rec.squeeze(0).squeeze(0).detach().cpu().numpy()
        rec_full_apact[20*i:20*i+80, 20*j:20*j+80] += rec

In [ ]:
fig = plt.figure(figsize=(16,6))
ax1 = plt.subplot(1,2,1)
plt.imshow(rec_full_apact, cmap='gray')
cax = fig.add_axes([ax1.get_position().x1+0.008, ax1.get_position().y0, 0.015, ax1.get_position().height])
cb = plt.colorbar(cax=cax)
cb.ax.tick_params(labelsize=10)
plt.title('Image', fontsize=12)

ax2 = plt.subplot(1,2,2)
plt.hist(rec_full_apact.ravel(), bins=150)
plt.title('Histogram', fontsize=12)

### Deconvolution with Single Body PSFs

In [ ]:
from re import S


def get_delays(R, v0, v1, n_delays, mode='uniform'):
    if mode == 'uniform':
        return np.linspace(0, (1-v0/v1) * R, n_delays)
    elif mode == 'quadric':
        return (1-v0/v1) * R * np.sqrt(np.linspace(0,1,n_delays))
    else:
        raise NotImplementedError()

n_delays = 16
delays = get_delays(R, v0, v1, n_delays, 'uniform')
print(delays)

recons = []
for d_delay in delays:
    recon = delay_and_sum(R_ring=0.05,
                          T_sample=1/(40e6), 
                          V_sound=v0,
                          Sinogram=sinogram[:,46:],
                          x_vec=kgrid.x_vec[:],
                          y_vec=kgrid.y_vec[:],
                          d_delay=d_delay,
                          ring_error=ring_error)
    recons.append(recon)

In [ ]:
recons = np.array(recons)
recons = recons[:,x_c-img_size//2:x_c+img_size//2, y_c-img_size//2:y_c+img_size//2]

In [ ]:
plt.figure(figsize=(13, 4))
for idx, d_delay in enumerate(delays):
    ax1 = plt.subplot(2,8,idx+1)
    plt.imshow(recons[idx,:,:], cmap='gray')
    plt.title('Delay={:.2f}mm'.format(d_delay*1e3), fontsize=12)
    plt.axis('off')
plt.tight_layout()

In [ ]:
l = 3.2e-3        
psfs, obss = [], []
recons = np.array(recons)
for i in range(25):
    for j in range(25):
        obs = torch.Tensor(recons[:,20*i:20*i+80, 20*j:20*j+80])
        obss.append(obs)
        
        x, y = (j-12)*l / 4, (12-i)*l / 4
        r, phi = np.sqrt(x**2 + y**2), np.arctan2(x, y)
        w_real = wavefront_real(R, r, phi, v0, v1)
        k2D, theta2D = get_fourier_coord(n_points=80, l=l, device='cpu')
        psf = []
        for id, delay in enumerate(delays):
            psf.append(PSF(theta2D, k2D, w_real, delay))
        psfs.append(torch.stack(psf, dim=0))

In [ ]:
wiener = Wiener_Batched(lam=0.03)
admm = ADMM_Batched(n_iters=16, lam=0.1, rho=0.1)

### Small Patch

In [ ]:
def get_r_C0(i, j, R, l, v0, v1):
    x, y = (j-12)*l / 4, (12-i)*l / 4
    r = np.sqrt(x**2 + y**2)
    C0 = np.maximum(0, (1-v0/v1) * R * (1 - (r**2)/(4*R**2)))
    return r, C0

i, j = 14, 22
r, C0 = get_r_C0(i, j, R, l, v0, v1)
print('r={:.2f}mm   C0={:.2f}mm'.format(r*1e3, C0*1e3))

In [ ]:
obs, psf = torch.Tensor(obss[i*25+j]), psfs[i*25+j]
obs, psf = obs.unsqueeze(0).to(device), psf.unsqueeze(0).to(device)
mean = obs.abs().mean()
with torch.no_grad():
    rec_wiener = wiener(obs/mean, psf) * mean
    rec_admm = admm(obs/mean, psf) * mean
obs = obs.squeeze(0).detach().cpu().numpy()
psf = psf.squeeze(0).detach().cpu().numpy()
rec_wiener = rec_wiener.squeeze(0).squeeze(0).detach().cpu().numpy()
rec_wiener /= np.abs(rec_wiener).mean()
rec_admm = rec_admm.squeeze(0).squeeze(0).detach().cpu().numpy()
rec_admm /= np.abs(rec_admm).mean()

In [ ]:
plt.figure(figsize=(15,10))
for i in range(n_delays):
    plt.subplot(5,8,i+1)
    plt.imshow(obs[i], cmap='gray')
    plt.title('Delay={:.2f}mm'.format(delays[i]*1e3), fontsize=13)
    plt.xticks([])
    plt.yticks([])
    if i == 0:
        plt.ylabel('Observation', fontsize=15)
    
    plt.subplot(5,8,i+17)
    plt.imshow(psf[i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    if i == 0:
        plt.ylabel('PSF', fontsize=15)
    
plt.subplot(5,8,3+2*n_delays)
plt.imshow(baseline[20*i:20*i+80, 20*j:20*j+80], cmap='gray')
plt.xticks([])
plt.yticks([])
plt.title('Conventional DAS', fontsize=15)

plt.subplot(5,8,4+2*n_delays)
plt.imshow(rec_wiener, cmap='gray')
plt.xticks([])
plt.yticks([])
plt.title('Wiener', fontsize=15)

plt.subplot(5,8,5+2*n_delays)
plt.imshow(rec_admm, cmap='gray')
plt.xticks([])
plt.yticks([])
plt.title('ADMM', fontsize=15)
    
plt.tight_layout()

In [ ]:
def gaussian_kernel(sigma, size):
    function = lambda x,y: np.exp(-((x-(size-1)/2)**2 + (y-(size-1)/2)**2) / (2*(sigma**2)))
    kernel = np.fromfunction(function, (size, size), dtype=float)
    return kernel / np.sum(kernel)

fwhm = 1.5e-3 # [m]
sigma = fwhm / 4e-5 / np.sqrt(2*np.log(2))
gaussian_window = gaussian_kernel(sigma, 80)

In [ ]:
def deconv_full_img(model, recons, gaussian_window, delays, attention='euclidean'):
    model.to(device)
    model.eval()
    
    rec_full = np.zeros_like(recons[0])
    for i in range(25):
        for j in range(25):
            obs, psf = obss[i*25+j], psfs[i*25+j]
            mean = obs.abs().mean()
            
            _, C0 = get_r_C0(i, j, R, l, v0, v1)
            weights = get_weights(C0, delays, attention)
            obs, psf = obs * np.sqrt(weights), psf * np.sqrt(weights) # Apply attention weights to different channels.
            obs, psf = obs.unsqueeze(0).to(device), psf.unsqueeze(0).to(device)
            with torch.no_grad():
                rec = model(obs/mean, psf) * mean
            rec = rec.squeeze(0).squeeze(0).detach().cpu().numpy()
            rec_full[20*i:20*i+80, 20*j:20*j+80] += rec * gaussian_window

    return rec_full

def get_weights(C0, delays, attention):
    """Calculates the weights for combining different delay channels in deconvolution

    Args:
        C0 (`float`): The zeroth order harmonic expansion coefficient of wavefront function.
        delays (`numpy.ndarray`): The array of delays used in delay-and-sum recontruction.
        attention (`string`): The type of attention weights (`['uniform', 'onehot', 'euclidean']`).

    Raises:
        NotImplementedError: The input `attention` type is not implemented.

    Returns:
        `numpy.ndarray`: The weights for different delay channels with shape `[n_delay, 1, 1]`.
    """
    n_delays = delays.shape[0]
    if attention == 'uniform':
        return np.ones([n_delays,1,1]) 
    elif attention == 'euclidean':
        distance = (delays.reshape([n_delays,1,1])-C0)**2
        weights = np.exp(distance) / np.exp(distance).sum()
        return weights / weights.sum() * n_delays
    elif attention == 'onehot':
        weights = np.zeros([n_delays,1,1])
        weights[np.argmin(np.abs(delays-C0))] = 1
        return weights
    else: 
        raise NotImplementedError('Attention type not implemented.')

rec_full_wiener = deconv_full_img(wiener, recons, gaussian_window, delays)
rec_full_admm = deconv_full_img(admm, recons, gaussian_window, delays)

Normalize image to the same intensity.

In [ ]:
# baseline, rec_full_apact, rec_full_wiener, rec_full_admm = [normalize(img) for img in [baseline, rec_full_apact, rec_full_wiener, rec_full_admm]]
baseline, rec_full_apact, rec_full_wiener, rec_full_admm = [standardize(img) for img in [baseline, rec_full_apact, rec_full_wiener, rec_full_admm]]

In [ ]:
print(baseline.min(), baseline.max(), baseline.std())
print(rec_full_apact.min(), rec_full_apact.max(), rec_full_apact.std())
print(rec_full_wiener.min(), rec_full_wiener.max(), rec_full_wiener.std())
print(rec_full_admm.min(), rec_full_admm.max(), rec_full_admm.std())

In [ ]:
patches = [(70, 200, 100, 200, 'blue'), (260, 325, 100, 200, 'red')]

fig = plt.figure(figsize=(15, 6.5))
gs = gridspec.GridSpec(5, 12)
norm = matplotlib.colors.Normalize(vmax=10, vmin=-9)
for idx, (rec_full, method) in enumerate(zip([baseline, rec_full_apact, rec_full_wiener], ['Conventional DAS', 'APACT', 'Wiener'])):
    ax = plt.subplot(gs[0:4,idx*4:(idx+1)*4])
    plt.imshow(rec_full, cmap='gray', norm=norm)
    plt.title(method, fontsize=18)
    plt.axis('off')
    for k, (x, y, h, w, color) in enumerate(patches):
        rect = plt.Rectangle((y, x), w, h, fill=False, edgecolor=color, linewidth=1.6, linestyle='--')
        ax.add_patch(rect)
        ax2 = plt.subplot(gs[4:5,idx*4+k*2:idx*4+(k+1)*2])
        plt.imshow(rec_full[x:x+h, y:y+w], cmap='gray', norm=norm)
        plt.xticks([])
        plt.yticks([])
        for loc in ['bottom', 'top', 'left', 'right']:
            ax2.spines[loc].set_color(color)
            ax2.spines[loc].set_linewidth(1.8)
            ax2.spines[loc].set_linestyle('--')
cax = fig.add_axes([ax.get_position().x1+0.01, ax.get_position().y0, 0.012, ax.get_position().height])
cb = plt.colorbar(cax=cax)

Compare different delay ranges.

In [ ]:
fig = plt.figure(figsize=(10, 5))
norm = matplotlib.colors.Normalize(vmax=1, vmin=0)
for i in range(2):
    ax = plt.subplot(1,2,i+1)
    if i == 0:
        delays = get_delays(R, v0, v1, n_delays, 'uniform')
        plt.title('Our range', fontsize=14)
    else:
        delays = np.arange(-8e-4, 8e-4, 1e-4)
        plt.title('APACT range', fontsize=14)
    rec_full = deconv_full_img(wiener, recons, gaussian_window, delays)
    rec_full = normalize(rec_full)
    print(rec_full.mean())
    plt.imshow(rec_full, cmap='gray', norm=norm)
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()

Compare different attention weights.

In [ ]:
# fig = plt.figure(figsize=(16, 10))
# for i, attention in enumerate(['uniform', 'euclidean', 'onehot']):
#     for j, mode in enumerate(['uniform', 'quadric']):
#         ax = plt.subplot(2,3,j*3+i+1)
#         delays = get_delays(R, v0, v1, n_delays, mode)
#         rec_full = deconv_full_img(wiener, recons, gaussian_window, delays, attention)
#         rec_full = standardize(rec_full)
#         plt.imshow(rec_full, cmap='gray', norm=norm)
#         if j == 0:
#             plt.title(attention, fontsize=20)
#         if i == 0:
#             plt.ylabel(mode, fontsize=20)
#         plt.xticks([])
#         plt.yticks([])
# plt.tight_layout()